In [3]:
import xarray as xr
import os
import glob
import pyaerocom as pya
from pyaerocom.io.readgridded import ReadGridded
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [4]:
# Setup directories and readers
# Control: files converted with EMEP bash script
# Convert: files converted with ReadEMEP

## Lustre
control_dir = '/lustre/storeA/project/aerocom/aerocom-users-database/AEROCOM-PHASE-III-2019/EMEPrv4.33-met2010_HIST/renamed'
convert_dir = '' # out_dir from EMEP_to_Aerocom

## Locally
# control_dir = '/home/eirikg/Desktop/pyaerocom/data/2020_AerocomHIST/control'
# convert_dir = '/home/eirikg/Desktop/pyaerocom/data/2020_AerocomHIST/processed'
data_id = "EMEPrv4.33-met2010"
control_reader = ReadGridded(data_dir=control_dir, data_id=data_id+'-control')
convert_reader = ReadGridded(data_dir=convert_dir, data_id=data_id+'-converted')

In [15]:
# Check which variables are available, print every variable missing from converted files
var_in_both = []

for variable in control_reader.vars_provided:
    if variable not in convert_reader.vars_provided:
        print('Variable: {} missing from converted files.'.format(variable))
    else:
        var_in_both.append(variable)

In [26]:
results = {}

def compare(control, convert, time=0):
    """
    Checks that the values for a given time in control and convert are relatively close (tol)
    """
    
    result = []
    diff = control.isel(time=time).values - convert.isel(time=time)
    mintest = float(diff.min()) == 0
    maxtest = float(diff.max()) == 0
    result = (mintest and maxtest)
    return result

In [27]:
# Loop through all variables and compare 
for variable in var_in_both:
    control = control_reader.read_var(variable).to_xarray().load()
    convert = convert_reader.read_var(variable).to_xarray().load()
    if variable[0:3] in ['dry', 'wet']: # Dry and wet deposition have been converted to kg m-2 s-1
#         continue
        # Will fail because of unit change
        result = compare(control, convert)
    else:
        result = compare(control, convert)
    results[variable] = result

In [30]:
# Print variables that failed the test
for key in results.keys():
    res = results[key]
    if res == False:
        print(key, ': control different from converted')

drybc : control different from converted
drydust : control different from converted
drynh4 : control different from converted
dryno3 : control different from converted
dryoa : control different from converted
dryso2 : control different from converted
dryso4 : control different from converted
dryss : control different from converted
wetbc : control different from converted
wetdust : control different from converted
wetnh4 : control different from converted
wetno3 : control different from converted
wetoa : control different from converted
wetoxn : control different from converted
wetrdn : control different from converted
wetso2 : control different from converted
wetso4 : control different from converted
wetsox : control different from converted
wetss : control different from converted
